In [3]:
#Georgios Cheirmpos - 3130230
#Machine Project 2

In [4]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
#preprocess image before EM
i_dont_always = np.array(Image.open('im.jpg'))/255.
w, h, d = i_dont_always.shape
image_array = np.reshape(i_dont_always, (w * h, d))
#rebuilding image from predictions
def recreate_image(image_array, i_dont_always, labels, w, h):
    d = image_array.shape[1]
    image = np.zeros((w, h, d))
    label_idx = 0
    cost_of_recreation = 0
    for i in range(w):
        for j in range(h):
            image[i][j] = image_array[labels[label_idx]]
            #caluclate cost
            cost_of_recreation += ((np.linalg.norm(image[i][j]-i_dont_always[i][j]))**2)/(w*h)
            label_idx += 1
    print("Total distance between pixels:" + str(cost_of_recreation))
    return image

In [5]:
def normal_dist(x, mean, sd):
    var = sd**2
    denom = (2*np.pi*var)**.5
    num = np.exp(-((x-mean)**2)/(2*var))
    return num/denom
    
def EM(X, K):
    #init basic param
    tol = 1e-5 #tolerance of convergence
    Jold = -np.inf
    maxIters = 100
    costs = []
    Pinit = np.ones(K)/K
    Minit = np.array([np.mean(X, axis=0),]*K) + np.random.randn(K,3)
    print(Minit)
    Sinit = np.random.rand(K)
    for it in range(maxIters): 
        # 1. E-step: responsibility = p(z_i = k | x_i, theta^(t-1))
        P_nominators = np.array([Pinit[k] * normal_dist(X, Minit[k], Sinit[k]) for k in range(K)])
        # responsibilites calculation
        gzki = np.array([[sum(P_nominators[k, i]/np.sum(P_nominators[:, i])) for k in range(K)] for i in range(len(X))])
        Nk = [sum(gzki[:, k]) for k in range(K)]
        # 2. M-step: Update m, sigma, pi values
        Mk_new = [((gzki[:, k].dot(X))/Nk[k]) for k in range(K)]
        print(Mk_new)
        sums = [sum(gzki[i,k]*((X[i] - Mk_new[k])**2) for i in range(len(X))) for k in range(K)]
        Sk_new = [sums[k]/Nk[k] for k in range(K)]
        Pk_new = [Nk[k]/len(X) for k in range(K)]
        # 3. Evaluation Step - Convergence check (Log Likelihood)
        Jnew = np.sum(np.log(np.sum(np.array([Pinit[k] * normal_dist(X, Minit[k], Sinit[k]) for k in range(K)]))))
        costs.append(Jnew)
        # 4. Update for loop 
        Pinit = Pk_new
        Minit = Mk_new
        Sinit = np.sqrt(Sk_new)
        print("Iteration #{}, Cost function value: {}".format(it, Jnew))
        print(np.abs(Jnew - Jold))
        #check for convergence or error
        if np.abs(Jnew - Jold) < tol or Jnew - Jold < 0:
            break
        Jold = Jnew  
         
    return Minit, np.argmax(gzki, axis = 1)

In [ ]:
#run for every K
for i in [1,2,4,8,16,32,64]:
    means, em_labels = EM(image_array, i)
    plt.imsave('images/ImageK'+str(i)+'_test.jpg', (recreate_image(np.array(means), i_dont_always, em_labels, w, h)))


[[ 0.06314666  0.60733007 -1.54201542]]
[array([0.2530868 , 0.18409021, 0.15108827])]
Iteration #0, Cost function value: 13.510408848496125
inf
[array([0.2530868 , 0.18409021, 0.15108827])]
Iteration #1, Cost function value: 14.179650727579741
0.6692418790836161
[array([0.2530868 , 0.18409021, 0.15108827])]
Iteration #2, Cost function value: 14.179650727579741
0.0
Total distance between pixels:0.17473886870074223
[[ 0.58970706  1.46041877 -1.16431546]
 [ 0.46508622  0.10072633  0.38221943]]
[array([0.28059381, 0.2047181 , 0.17142506]), array([0.18687891, 0.13444001, 0.10213872])]
Iteration #0, Cost function value: 13.581272732760901
inf
[array([0.32181266, 0.24397329, 0.20390178]), array([0.1372029, 0.0831168, 0.0620354])]
Iteration #1, Cost function value: 14.301364626187764
0.7200918934268632
[array([0.39629573, 0.30851809, 0.25897214]), array([0.1192249 , 0.06778356, 0.05024583])]
Iteration #2, Cost function value: 14.538127613752977
0.2367629875652124
[array([0.49274465, 0.39255304

Iteration #8, Cost function value: 15.05065104468458
0.001381492862908118
[array([0.20396328, 0.1405091 , 0.10604816]), array([0.79923806, 0.6721958 , 0.5890386 ]), array([0.28525934, 0.20617366, 0.15732139]), array([0.69015463, 0.56282594, 0.47920245]), array([0.67738851, 0.5506026 , 0.46739363]), array([0.11031308, 0.05962465, 0.0439261 ]), array([0.70863661, 0.5805935 , 0.49644792]), array([0.67750174, 0.5507109 , 0.46749811])]
Iteration #9, Cost function value: 15.050369754021592
0.00028129066298809846
Total distance between pixels:0.019763613467521618
[[ 1.64697824  0.65617123  0.57781396]
 [-0.12868849  3.0213754   0.21584543]
 [-1.32902549 -1.00701557 -0.10130113]
 [ 0.36860062  0.96936111  0.38200806]
 [ 1.9673489   0.86575531  0.05765307]
 [-1.49060665 -0.91972889 -0.43177152]
 [-0.81558447  1.62939066  0.07027549]
 [ 0.9407528   0.78224906  0.69732041]
 [-0.83657233 -0.02413643  1.65408177]
 [ 0.76409709 -0.3083252   1.90586442]
 [-1.07103361  1.17994093  0.16722427]
 [-0.780

Iteration #8, Cost function value: 15.052551225573525
0.002261991390293261
[array([0.62094437, 0.49535674, 0.41305065]), array([0.12211297, 0.07090581, 0.05254035]), array([0.15651023, 0.10090671, 0.07546707]), array([0.61193696, 0.48682336, 0.40489304]), array([0.20098239, 0.1380943 , 0.1041536 ]), array([0.16361045, 0.10691089, 0.08007798]), array([0.15120974, 0.09639887, 0.07201095]), array([0.7926079 , 0.66460049, 0.58061503]), array([0.45279613, 0.34191555, 0.27029149]), array([0.53758249, 0.41722879, 0.33895198]), array([0.17763024, 0.11868361, 0.08914306]), array([0.79221022, 0.66422613, 0.58024678]), array([0.13296581, 0.08062019, 0.05994865]), array([0.37835427, 0.28018007, 0.21720408]), array([0.29517902, 0.21418812, 0.16366932]), array([0.10348092, 0.0530049 , 0.03886614])]
Iteration #9, Cost function value: 15.05288296045731
0.0003317348837850176
[array([0.63031119, 0.50431135, 0.42167287]), array([0.1235277 , 0.07213977, 0.05347939]), array([0.16160956, 0.10517917, 0.07875